# Ternary Tree Class

In [1]:
"""Ternary Tree fermion to qubit mappings"""

import numpy as np
from base import FermionQubitEncoding
from ternary_tree_node import TTNode, node_sorter
from utils import symplectic_product, icount_to_sign
from functools import cached_property
import logging

logger = logging.getLogger(__name__)

class TernaryTree(FermionQubitEncoding):
    def __init__(
        self,
        one_e_coeffs: np.ndarray,
        two_e_coeffs: np.ndarray,
        qubits: set = None,
        root_node: TTNode = TTNode(),
        enumeration_scheme: dict[str, tuple[int, int]] | None = None,
    ):
        super().__init__(one_e_coeffs, two_e_coeffs, qubits)
        self.root = root_node
        self.root.label = ""
        self.enumeration_scheme = enumeration_scheme
        self.n_qubits = self.one_e_coeffs.shape[1]

    def default_enumeration_scheme(self):
        node_strings = self.root.child_strings
        enumeration_scheme = {node: (None, None) for node in node_strings}
        enumeration_scheme = {
            k: {"mode": i, "qubit": i} for i, k in enumerate(enumeration_scheme)
        }
        return enumeration_scheme

    def as_dict(self):
        return self.root.as_dict()

    def add_node(self, node_string: str):
        node_string = node_string.lower()
        valid_string = np.all([char in ["x", "y", "z"] for char in node_string])
        if not valid_string:
            raise ValueError("Branch string can only contain x,y,z")

        node = self.root
        for char in node_string:
            if isinstance(getattr(node, char), TTNode):
                node = getattr(node, char)
            else:
                node = node.add_child(char, node.label + char)
        return self

    @property
    def branch_operator_map(self):
        if self.enumeration_scheme is None:
            raise ValueError("enumeration scheme not set")

        branches = self.root.branch_strings

        nodes = self.root.child_strings
        node_indices = {node: i for i, node in enumerate(nodes)}

        branch_operator_map = {}
        for branch in branches:
            branch_operator_map[branch] = ["I"] * self.n_qubits
            node = self.root
            for char in branch:
                node_index = node_indices[node.label]
                # qubit_index = self.enumeration_scheme[node.label]["qubit"]
                branch_operator_map[branch][node_index] = char.upper()
                node = getattr(node, char, None)
            branch_operator_map[branch] = "".join(branch_operator_map[branch])

        return branch_operator_map

    @property
    def string_pairs(self):
        """Return the pair of branch strings which correspond to each node.

        Returns:
            dict[str, tuple(str,str)]: A dictionary of all node labels, j,  with branch strings (2j, 2j+1).
        """
        node_set = self.root.child_strings

        pairs = {}
        for node_string in node_set:
            node = self.root
            for char in node_string:
                node = getattr(node, char)

            x_string = node_string + "x"
            y_string = node_string + "y"
            if x_string in node_set:
                while True:
                    x_string += "z"
                    if x_string not in node_set:
                        break

            if y_string in node_set:
                while True:
                    y_string += "z"
                    if y_string not in node_set:
                        break

            if x_string.count("y") % 2 == 0:
                pairs[node.label] = x_string, y_string
            elif y_string.count("y") % 2 == 0:
                pairs[node.label] = y_string, x_string

        return pairs

    def _build_symplectic_matrix(self):
        # If there isn't one provided, assume the naive one
        if self.enumeration_scheme is None:
            self.enumeration_scheme = self.default_enumeration_scheme()

        pauli_string_map = self.branch_operator_map
        
        symplectic = np.zeros((2 * self.n_qubits, 2 * self.n_qubits), dtype=np.byte)
        ipowers = np.zeros((2 * self.n_qubits), dtype=np.uint8)
        for node, operators in self.string_pairs.items():
            for offset, operator in enumerate(operators):
                operator = pauli_string_map[operator]
                operator = np.array(list(operator))
                # If the string is X or Y then assign 1
                term_ipower, symplectic_term = self._pauli_to_symplectic(operator)
                fermion_mode = self.enumeration_scheme[node]["mode"]
                ipowers[2 * fermion_mode + offset] = term_ipower
                symplectic[2 * fermion_mode + offset] = symplectic_term
        return ipowers, symplectic

    def JordanWigner(self):
        new_tree = TernaryTree(
            one_e_coeffs=self.one_e_coeffs,
            two_e_coeffs=self.two_e_coeffs,
            qubits=self.qubits,
            root_node=TTNode(),
        )
        new_tree.add_node("z" * (self.n_qubits - 1))
        new_tree.enumeration_scheme = new_tree.default_enumeration_scheme()
        return new_tree

    def JW(self):
        return self.JordanWigner()

    def ParityEncoding(self):
        new_tree = TernaryTree(
            one_e_coeffs=self.one_e_coeffs,
            two_e_coeffs=self.two_e_coeffs,
            qubits=self.qubits,
            root_node=TTNode(),
        )
        new_tree.add_node("x" * (len(self.qubits) - 1))
        new_tree.enumeration_scheme = new_tree.default_enumeration_scheme()
        return new_tree

    def BravyiKitaev(self):
        new_tree = TernaryTree(
            one_e_coeffs=self.one_e_coeffs,
            two_e_coeffs=self.two_e_coeffs,
            qubits=self.qubits,
            root_node=TTNode(),
        )
        branches = ["x"]
        # one is used for root, which is defined
        remaining_qubits = len(self.qubits) - 1
        while remaining_qubits > 0:
            new_branches = set()
            for item in branches:
                if remaining_qubits > 0:
                    new_tree.add_node(item)
                    remaining_qubits -= 1
                else:
                    break

                new_branches.add(item + "x")
                new_branches.add(item + "z")
            branches = sorted(list(new_branches), key=node_sorter)
        new_tree.enumeration_scheme = new_tree.default_enumeration_scheme()
        return new_tree

    def BK(self):
        return self.BravyiKitaev()

    def JKMN(self):
        new_tree = TernaryTree(
            one_e_coeffs=self.one_e_coeffs,
            two_e_coeffs=self.two_e_coeffs,
            qubits=self.qubits,
            root_node=TTNode(),
        )
        branches = ["x", "y", "z"]
        # one is used for root which is defined
        remaining_qubits = len(self.qubits) - 1
        while remaining_qubits > 0:
            new_branches = set()
            for item in branches:
                if remaining_qubits > 0:
                    new_tree.add_node(item)
                    remaining_qubits -= 1
                else:
                    break

                new_branches.add(item + "x")
                new_branches.add(item + "y")
                new_branches.add(item + "z")
            branches = sorted(list(new_branches), key=node_sorter)
        new_tree.enumeration_scheme = new_tree.default_enumeration_scheme()
        return new_tree

In [ ]:
import pickle
with open("../tests/hamiltonians/ones_spinorb.pkl","rb") as file:
    nbed_ones = pickle.load(file)
with open("../tests/hamiltonians/twos_spinorb.pkl","rb") as file:
    nbed_twos = pickle.load(file)



[[-3.44519546e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   5.82789853e-01  0.00000000e+00  1.22739269e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.78336822e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.94394367e-01  0.00000000e+00]
 [ 0.00000000e+00 -3.44510678e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -5.78552627e-01  0.00000000e+00  1.04183481e-08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.88260278e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.92345861e-01]
 [ 0.00000000e+00  0.00000000e+00 -3.44407428e+01  0.00000000e+00
  -1.208

In [15]:
mytt = TernaryTree(nbed_ones,nbed_twos) 

In [21]:
nbed_ones.shape

(28, 28)

In [25]:
from random import shuffle

op_list = [*range(nbed_ones.shape[1])]
shuffle(op_list)

op_list2 = [*range(nbed_ones.shape[1])]
shuffle(op_list2)

print(op_list)
print(op_list2)

[14, 6, 5, 26, 11, 22, 18, 27, 24, 2, 9, 10, 7, 25, 13, 1, 20, 17, 0, 16, 21, 3, 4, 12, 8, 15, 23, 19]
[14, 2, 17, 10, 5, 4, 19, 1, 8, 22, 9, 23, 7, 21, 0, 13, 20, 6, 25, 26, 18, 24, 11, 15, 3, 27, 16, 12]


In [27]:
qham1 = mytt.JW().to_qubit_hamiltonian()
qham2 = mytt.JW().to_qubit_hamiltonian(mode_op_map={i:op_list[i] for i in range(nbed_twos.shape[1])})
qham3 = mytt.JW().to_qubit_hamiltonian(mode_op_map={i:op_list2[i] for i in range(nbed_twos.shape[1])})

In [30]:
import pickle

with open("qham1.plk","wb") as file:
    pickle.dump(qham1, file)
with open("qham2.plk","wb") as file:
    pickle.dump(qham2, file)
with open("qham3.plk","wb") as file:
    pickle.dump(qham3, file)

# New Functions

In [2]:
def edge_operator_map(encoding: FermionQubitEncoding) -> tuple[dict, dict]:
    """Build a map of operators in the full hamiltonian to their constituent majoranas.
    
    """
    majorana_symplectic = encoding._build_symplectic_matrix()[1]

    icount, sym_products = encoding.symplectic_product_map
    edge_map = {}

    n_modes = majorana_symplectic.shape[1]//2

    for m in range(n_modes):
        for n in range(n_modes):
            # if self.one_e_coeffs[m,n] == 0:
                # continue
            
            factor = 0.25  # if m == n else 0.25
            # (gamma_2m -i gamma_2m+1)(gamma_2n +i gamma_2n+1)
            first_term = sym_products[(2 * m, 2 * n)]
            second_term = sym_products[(2 * m, 2 * n + 1)]
            third_term = sym_products[(2 * m + 1, 2 * n)]
            fourth_term = sym_products[(2 * m + 1, 2 * n + 1)]

            factors = factor * (
                icount_to_sign(icount[2 * m, 2 * n]), 
                icount_to_sign(icount[2 * m, 2 * n+1]+1),
                icount_to_sign(icount[2 * m+1, 2 * n]+3), 
                icount_to_sign(icount[2 * m+1, 2 * n+1]), 
                                                )
            terms = [first_term, second_term, third_term, fourth_term]

            if m <= n:
                # edge_map[(m,n)] = {term: factor for term, factor in zip(terms, factors)}
                
            # The other way round will always come second!
            else:
                for t, f in zip(terms, factors):
                    edge_map[(n,m)][t] += f
                    if edge_map[(n,m)][t] == 0:
                        edge_map[(n,m)].pop(t)

    weights = np.zeros((n_modes, n_modes))
    for k,v in edge_map.items():
        x_block, z_block = np.hsplit(np.vstack([np.fromstring(op[1:-1], dtype=np.uint8, sep=" ") for op in v.keys()]),2)
        
        mean_weight = np.mean(np.sum(np.bitwise_or(x_block, z_block), axis=1) * [factor for factor in v.values()])
        weights[k[0], k[1]] = mean_weight
        weights[k[1], k[0]] = mean_weight

    return edge_map, weights

IndentationError: expected an indented block after 'if' statement on line 32 (1722025040.py, line 36)

In [ ]:
mydict = {"a":1}

: 